In [1]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

In [2]:
pd.options.display.max_rows = 10
sns.set()
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'sans-serif']

In [3]:
%matplotlib notebook

In [5]:
def to_number(cell):
    if not isinstance(cell, str):
        return cell
    _ = re.findall(r'\d+', cell)
    if _:
        return int(_[0])


df = pd.read_excel("购地-地块公示.xlsx", index_col="企业编号")
df['出让年限'] = df['出让年限'].map(to_number)
df

,行政区,时间,公示日期,土地用途,土地面积（公顷）,出让年限,成交价（万元）,土地使用条件
企业编号,,,,,,,,
4235,杭州市,2013-09-27,2013年09月27日至2013年10月07日,工业用地,0.746100,50.0,360.00000,NaN
4236,安徽省 > 淮南市 > 淮南市本级,2011-12-30,2011年12月30日至2012年01月10日,工业用地,7.141500,50.0,2406.69000,NaN
4237,江西省 > 南昌市 > 南昌县,2012-06-28,2012年06月28日至2012年07月08日,工业用地,0.674700,50.0,345.67000,"工业用地(医疗器械),容积率大于等于0.7,建筑密度大于等于30%,绿地率小于等于20%"
4238,广东省 > 深圳市 > 深圳市本级,2018-07-04,2018年07月04日至2018年07月11日,工业用地,0.593388,30.0,27100.00000,NaN
4242,广东省 > 珠海市 > 珠海市本级,2012-05-18,2012年05月17日至2012年05月27日,工业用地,1.146056,50.0,550.10688,NaN
...,...,...,...,...,...,...,...,...
4520,吴江市,2016-07-19,2016年07月20日至2016年07月25日,工业用地,4.187240,50.0,1415.28712,净地;
4522,江苏省 > 南通市 > 如东县,2017-10-30,2017年10月30日至2017年11月03日,工业用地,3.375900,50.0,962.13150,净地;
4523,中山市,2014-07-23,NaN,工矿仓储用地,5.650500,NaN,NaN,已开发


In [6]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days

In [7]:
time = df[['时间']].applymap(to_day).groupby('企业编号').mean().T
time = time.reindex(columns=list(range(4001,4526)))
time

企业编号,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,...,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525
时间,NaN,NaN,NaN,-2724.0,-2505.0,NaN,-2745.0,NaN,NaN,NaN,...,-2484.0,-2636.0,-1412.5,-850.8,-2150.75,NaN,-491.0,-1686.0,-414.0,NaN


In [9]:
lst = df['土地用途'].value_counts()[df['土地用途'].value_counts() == 1].index


def to_land_use(cell):
    if not isinstance(cell, str):
        return cell
    if cell not in lst:
        return cell
    
    
land_use = pd.get_dummies(df['土地用途'].map(to_land_use), prefix='土地用途').groupby('企业编号').sum().T
land_use = land_use.reindex(columns=list(range(4001,4526))).fillna(0).reindex(['土地用途_05', '土地用途_051', '土地用途_054', '土地用途_05、07', '土地用途_06', '土地用途_061',
       '土地用途_07', '土地用途_071', '土地用途_073', '土地用途_100', '土地用途_12',
       '土地用途_中低价位、中小套型普通商品住房用地', '土地用途_交通运输用地', '土地用途_仓储用地', '土地用途_住宅用地',
       '土地用途_住宿餐饮用地', '土地用途_公共租赁住房用地', '土地用途_公共管理与公共服务用地', '土地用途_公共设施用地',
       '土地用途_公路用地', '土地用途_其他住房用地', '土地用途_其他商服用地', '土地用途_其他土地',
       '土地用途_其他普通商品住房用地', '土地用途_医卫慈善用地', '土地用途_商务金融用地', '土地用途_商服用地',
       '土地用途_土地面积(公顷)', '土地用途_工业用地', '土地用途_工矿仓储用地', '土地用途_廉租住房用地',
       '土地用途_批发零售用地', '土地用途_文体娱乐用地', '土地用途_新闻出版用地', '土地用途_机关团体用地', '土地用途_机场用地',
       '土地用途_殡葬用地', '土地用途_水工建筑用地', '土地用途_水库水面', '土地用途_港口码头用地', '土地用途_特殊用地',
       '土地用途_科教用地', '土地用途_空闲地', '土地用途_管道运输用地', '土地用途_经济适用住房用地', '土地用途_街巷用地',
       '土地用途_采矿用地', '土地用途_铁路用地', '土地用途_风景名胜设施用地', '土地用途_高档住宅用地']).fillna(0)
land_use

企业编号,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,...,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525
土地用途_05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
土地用途_051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
土地用途_054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
土地用途_05、07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
土地用途_06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
土地用途_街巷用地,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
土地用途_采矿用地,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
土地用途_铁路用地,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
土地用途_风景名胜设施用地,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
land_area_hectare = df[['土地面积（公顷）']].groupby('企业编号').sum().T
land_area_hectare = land_area_hectare.reindex(columns=list(range(4001,4526)))
land_area_hectare

企业编号,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,...,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525
土地面积（公顷）,NaN,NaN,NaN,2.4771,15.4012,NaN,3.32,NaN,NaN,NaN,...,3.158712,4.7635,6.81005,98.8771,7.94279,NaN,3.3759,5.6505,6.327854,NaN


In [11]:
transfer_period = df[['出让年限']].groupby('企业编号').mean().T
transfer_period = transfer_period.reindex(columns=list(range(4001,4526)))
transfer_period

企业编号,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,...,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525
出让年限,NaN,NaN,NaN,50.0,50.0,NaN,50.0,NaN,NaN,NaN,...,50.0,50.0,50.0,56.666667,50.0,NaN,50.0,NaN,30.0,NaN


In [12]:
transaction_price_ten_thousand_yuan = df[['成交价（万元）']].groupby('企业编号').sum().T
transaction_price_ten_thousand_yuan = transaction_price_ten_thousand_yuan.reindex(columns=list(range(4001,4526)))
transaction_price_ten_thousand_yuan

企业编号,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,...,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525
成交价（万元）,NaN,NaN,NaN,832.3056,4528.0,NaN,607.56,NaN,NaN,NaN,...,1318.1199,647.836,4554.726,58141.0,2468.50024,NaN,962.1315,0.0,2848.0,NaN


In [13]:
pd.concat([time, land_use, land_area_hectare, transfer_period, transaction_price_ten_thousand_yuan]).to_pickle('购地-地块公示testing.pickle')